# **Machine Learning**

#### Realizaremos un modelo de recomendación de películas en base a nuestro set de datos de streaming

In [29]:
# Librerias a utilizar
import pandas as pd
import re
from sklearn import preprocessing

In [30]:
# Importamos el data set "df_streaming" anteriormente tratado en el archivo (ETL.ipynb)
df = pd.read_csv("Datasets/df_streaming_completo.csv")

## `Transformaciones` a realizar :

+ Eliminaremos las columnas "country", "date_added", "director", "cast", "show_id".

+ Solucionaremos un error de datos de la columna "rating".
+ Imputaremos los valores faltantes en "duration_type", "duration_int", y "description".
+ Aplicaremos dummies a la columna "listed_in" (que contiene los géneros) y a la columna "type".
+ Aplicaremos un pre-procesamiento a la columna "rating"para convertirlas a numéricas.


##### `Aclaración` : Dado la corta experiencia en ingenieria de datos y ML, se decide no utilizar ciertas columnas, eliminándolas del análisis. Estas mismas podrían tener algún valor predictivo para el armado del modelo, pero esto requiere más tiempo y cuidado. Y este proyecto sigue una metodología MVP buscando tener un "Minimum Viable Product" lo más rápido posible

In [31]:
# También dropeamos las columnas "duration_int","duration_type" ya que en el paso siguiente volvemos a hacer el split
df.drop(["country","date_added","director","cast","show_id","duration_int","duration_type"],axis=1 , inplace=True)

In [32]:
# Iterar en el dataframe
for indice, fila in df.iterrows():
    match = re.search(r'\d+\s*min', fila["rating"])
    match2 = re.search(r'\d+\s*season', fila["rating"])
    # Imputamos en columna duration esas filas que están en rating
    if match:
        df.at[indice, "duration"] = fila["rating"]
    if match2:
        df.at[indice, "duration"] = fila["rating"]

# Reemplazamos los valores erróneos por "sin dato"
df.loc[df['rating'].str.contains(r'\d+\s*min'), 'rating'] = 'sin dato'
df.loc[df['rating'].str.contains(r'\d+\s*season'), 'rating'] = 'sin dato'

# Utilizamos el método str.split para separar el campo "duration" a través del espacio 
df[["duration_int", "duration_type"]] = df["duration"].str.split(" ", n=1, expand=True)

# Unificamos "seasons" y "season" en la columna duration_type
df.loc[df['duration_type']=='seasons', 'duration_type'] = 'season'

# Eliminamos del modelo la columna "duration"
df.drop("duration",inplace=True,axis=1)

In [33]:
# Trabajaremos con "duration_int"
# primero rellenamos con 0 y cambiamos el tipo de dato
df["duration_int"].fillna(0,inplace=True)
df["duration_int"] = df["duration_int"].astype(int) 

# Ahora podemos imputarle los valores faltantes a través de la mediana para ser robusta a valores atípicos
median_duration = df["duration_int"].median()

# Por lo que se pudo observar en el EDA, los valores faltantes en "duration_int" son solo de películas
df["duration_int"].fillna(median_duration,inplace=True)

# Entonces en "duration_type" se imputa el valor str "min" solo donde sea película
df["duration_type"].fillna(df["type"].where(df["type"]=="tv Show", other="min"), inplace=True)


In [34]:
# Se puede observar de que dan distintos números los datos
df["type"].value_counts()

movie      16481
tv show     6517
Name: type, dtype: int64

In [35]:
df["duration_type"].value_counts() 

min       16419
season     6579
Name: duration_type, dtype: int64

In [36]:
# Aquí podemos observar que el "type" es movie, pero duration_int y duration_type dicen que es season. Para corroborar buscamos alguno de los titulos en google.
df[(df["type"] == "movie") & (df["duration_type"] != "min")]

,type,title,release_year,rating,listed_in,description,id,score,duration_int,duration_type
11168,movie,yashahime: princess half-demon,2020,sin dato,anime,long-lost twins towa and setsuna reunite after...,hs51,3.56,2,season
11381,movie,trolls: trollstopia,2020,sin dato,"family, kids",trolls: trollstopia! is the next chapter in th...,hs264,3.57,4,season
11589,movie,this way up,2019,sin dato,"comedy, drama, international","this way up is a comedy drama about moving on,...",hs472,3.54,2,season
11919,movie,everything's gonna be okay,2020,sin dato,"comedy, drama",australian twenty-something nicholas is left t...,hs802,3.58,2,season
11965,movie,juda,2017,sin dato,"action, adventure, crime",juda is a low-life gambler hustling a living i...,hs848,3.48,2,season
...,...,...,...,...,...,...,...,...,...,...
14026,movie,the wrong mans,2013,sin dato,"action, adventure, comedy",lowly office workers sam (matthew baynton) and...,hs2909,3.48,2,season
14044,movie,the hotwives of orlando,2014,sin dato,"comedy, sitcom",the reality genre parody examines the glamorou...,hs2927,3.54,1,season
14069,movie,all saints (1998),1998,sin dato,"drama, international",all saints is an australian medical drama focu...,hs2952,3.60,12,season
14073,movie,getting on,2009,sin dato,"comedy, international, sitcom",care for the elderly is the least glamorous ar...,hs2956,3.56,3,season


In [37]:
# Y llegamos a la conclusión de que los valores erroneos son los de "type", por lo que lo reemplazamos por tv show
df.loc[(df["type"] == "movie") & (df["duration_type"] != "min"), "type"] = "tv show"

In [38]:
# Comprobamos y se puede observar de que ahora están correctos los valores
df["type"].value_counts()

movie      16419
tv show     6579
Name: type, dtype: int64

In [39]:
df["duration_type"].value_counts() 

min       16419
season     6579
Name: duration_type, dtype: int64

In [40]:
# Separamos los valores en la columna "listed_in"
split_values = df["listed_in"].str.split(", ", expand=True)

# Aplicamos get_dummies a todas las columnas separadas
dummies = pd.get_dummies(split_values, prefix="genero", prefix_sep="_")

# Agrupamos las columnas por nombre y las sumamos
dummies = dummies.groupby(dummies.columns, axis=1).sum()

# Concatenamos las variables dummies con la base de datos original
df = pd.concat([df, dummies], axis=1)

# Todavia nos quedan muchos géneros por separado, dando un total de muchas columnas, entonces lo que hacemos es agruparlas de forma mas generalizada

# Seleccionamos las columnas a combinar
cols_to_combine = ["genero_crime", "genero_crime tv shows","genero_police/cop","genero_spy/espionage","genero_unscripted","genero_action & adventure", "genero_action", "genero_adventure", "genero_action-adventure","genero_military and war","genero_tv action & adventure","genero_horror","genero_horror movies","genero_tv horror","genero_tv mysteries"]
cols_to_combine1 = ["genero_fantasy","genero_tv sci-fi & fantasy","genero_sci-fi & fantasy","genero_science & nature tv","genero_science fiction","genero_adult animation", "genero_animation", "genero_anime", "genero_anime features","genero_anime series"]
cols_to_combine3 = ["genero_and culture","genero_and culture", "genero_classic & cult tv", "genero_classic movies", "genero_classics","genero_cult movies","genero_western","genero_survival","genero_animals & nature", "genero_documentaries", "genero_disaster", "genero_documentary","genero_docuseries","genero_historical","genero_history","genero_mystery","genero_black stories","genero_coming of age"]
cols_to_combine4 = ["genero_science & technology","genero_anthology", "genero_biographical","genero_soap opera / melodrama","genero_music videos and concerts","genero_dance","genero_arthouse", "genero_arts","genero_music","genero_concert film","genero_music videos and concerts","genero_musical","genero_music & musicals"]
cols_to_combine6 = ["genero_variety","genero_teen tv shows","genero_teen","genero_talk show and variety","genero_lgbtq+","genero_lgbtq movies","genero_lgbtq","genero_latino","genero_children & family movies", "genero_cartoons", "genero_buddy", "genero_family","genero_kids","genero_kids' tv","genero_korean tv shows","genero_independent movies","genero_late night"]
cols_to_combine7 = ["genero_british tv shows", "genero_international", "genero_international movies", "genero_international tv shows"]
cols_to_combine9 = ["genero_travel","genero_cooking & food", "genero_faith & spirituality","genero_faith and spirituality","genero_fitness","genero_health & wellness","genero_lifestyle","genero_lifestyle & culture","genero_medical"]
cols_to_combine10 = ["genero_comedies", "genero_comedy","genero_sketch comedy","genero_stand-up comedy","genero_stand-up comedy & talk shows","genero_tv comedies"]
cols_to_combine11 = ["genero_romance","genero_romantic comedy","genero_romantic movies","genero_romantic tv shows","genero_dramas", "genero_drama","genero_suspense","genero_thriller","genero_thrillers","genero_tv dramas","genero_tv thrillers"]
cols_to_combine13 = ["genero_tv shows","genero_young adult audience","genero_talk show","genero_superhero","genero_stand up","genero_sports movies","genero_sports","genero_special interest","genero_spanish-language tv shows","genero_sitcom","genero_series","genero_movies","genero_entertainment","genero_game show / competition","genero_game shows","genero_news","genero_parody","genero_reality","genero_reality tv"]

# Combinamos las columnas utilizando el método sum() de pandas
df["accion"] = df[cols_to_combine].sum(axis=1)
df["animacion"] = df[cols_to_combine1].sum(axis=1)
df["documentales"] = df[cols_to_combine3].sum(axis=1)
df["arte y ciencia"] = df[cols_to_combine4].sum(axis=1)
df["familia"] = df[cols_to_combine6].sum(axis=1)
df["internacional"] = df[cols_to_combine7].sum(axis=1)
df["vida"] = df[cols_to_combine9].sum(axis=1)
df["comedia"] = df[cols_to_combine10].sum(axis=1)
df["drama"] = df[cols_to_combine11].sum(axis=1)
df["entretenimiento"] = df[cols_to_combine13].sum(axis=1)

# Eliminamos las columnas antiguas
df.drop(columns=cols_to_combine, axis=1,inplace=True)
df.drop(columns=cols_to_combine1, axis=1,inplace=True)
df.drop(columns=cols_to_combine3, axis=1,inplace=True)
df.drop(columns=cols_to_combine4, axis=1,inplace=True)
df.drop(columns=cols_to_combine6, axis=1,inplace=True)
df.drop(columns=cols_to_combine7, axis=1,inplace=True)
df.drop(columns=cols_to_combine9, axis=1,inplace=True)
df.drop(columns=cols_to_combine10, axis=1,inplace=True)
df.drop(columns=cols_to_combine11, axis=1,inplace=True)
df.drop(columns=cols_to_combine13, axis=1,inplace=True)

df["accion"] = df["accion"].replace({2 : 1, 3 : 1  })
df["animacion"] = df["animacion"].replace({2 : 1, 3 : 1  })
df["documentales"] = df["documentales"].replace({2 : 1, 3 : 1  })
df["arte y ciencia"] = df["arte y ciencia"].replace({2 : 1, 3 : 1  }) 
df["familia"] = df["familia"].replace({2 : 1, 3 : 1  })
df["internacional"] = df["internacional"].replace({2 : 1, 3 : 1  })
df["vida"] = df["vida"].replace({2 : 1, 3 : 1  })
df["comedia"] = df["comedia"].replace({2 : 1, 3 : 1  })
df["drama"] = df["drama"].replace({2 : 1, 3 : 1  })
df["entretenimiento"] = df["entretenimiento"].replace({2 : 1, 3 : 1  })

# dummies para type
dummies2 = pd.get_dummies(df['type'])
df = pd.concat([df, dummies2], axis=1)

# Borramos las columnas a las cuales se les aplico las dummies
df.drop(["type","listed_in"],axis=1,inplace=True)

In [41]:
# Aplicamos un preprocesamiento a "rating" y a "id" para convertirlos a numéricos
le = preprocessing.LabelEncoder()
df["rating"] = le.fit_transform(df["rating"])

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            22998 non-null  object 
 1   release_year     22998 non-null  int64  
 2   rating           22998 non-null  int32  
 3   description      22994 non-null  object 
 4   id               22998 non-null  object 
 5   score            22998 non-null  float64
 6   duration_int     22998 non-null  int32  
 7   duration_type    22998 non-null  object 
 8   accion           22998 non-null  int64  
 9   animacion        22998 non-null  int64  
 10  documentales     22998 non-null  int64  
 11  arte y ciencia   22998 non-null  int64  
 12  familia          22998 non-null  int64  
 13  internacional    22998 non-null  int64  
 14  vida             22998 non-null  int64  
 15  comedia          22998 non-null  int64  
 16  drama            22998 non-null  int64  
 17  entretenimie

In [43]:
# Exportamos el df con las transformaciones
df2 = df.copy()
df2.drop(["title","description","id","duration_type"],axis=1)
df2.to_csv("df_modelo.csv",index=False)

#### Comenzamos con el modelado

#### Este es el primer modelo que creamos, en rasgos generales no es la mejor opción ya que es muy pesado. Se recomienda no ejecutarlo. 
###### Mas abajo se encuentra un mejor modelo

In [44]:
""""

# Es necesario convertir todas las columnas a str para el modelo
for col in df.columns:
    df[col] = df[col].astype(str)

# Importar librerías
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear matriz de características
tfidf = TfidfVectorizer(stop_words='english')

# Combinar las columnas deseadas en una sola columna
df['features'] = df['score'] +' '+ df['rating'] +' '+ df['accion'] +' '+ df['drama'] +' '+ df['animacion'] +' '+ df['documentales'] +' '+ df['arte y ciencia'] +' '+ df['familia'] +' '+ df['internacional'] +' '+ df['vida'] +' '+ df['comedia'] +' '+ df['entretenimiento'] +' '+ df['comedia']+' '+ df['movie'] +' '+ df['tv show']  +' '+ df['release_year'] +' '+ df['duration_int'] +' '+ df['title'] 

# Crear la matriz de características a partir de la columna combinada
tfidf_matrix = tfidf.fit_transform(df['features'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Función para obtener recomendaciones
def get_recommendations(title, cosine_sim=cosine_sim, df=df):
    # Obtener el índice del elemento que coincide con el título
    idx = df[df['title'] == title].index[0]
    # Obtener las puntuaciones de similitud de coseno del elemento con todos los elementos
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Ordenar los elementos según su puntuación de similitud de coseno
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Obtener los índices de los elementos más similares
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    # Devolver los títulos de los elementos más similares
    peliculas = df[['title',"score"]].iloc[movie_indices]
    # Ordenamos según "score" descendente
    peliculas = peliculas.sort_values(by="score",ascending = False)
    # Convertimos a lista y solo tomamos el titulo
    peliculas = list(peliculas["title"])
    # Retornamos un diccionario con una lista de las 5 películas ordenadas de forma descendente según el score
    return {'recomendacion': peliculas[:5]}
# Obtener recomendaciones para una película específica
get_recommendations("marvel studios' iron man 2")

"""

'"\n\n# Es necesario convertir todas las columnas a str para el modelo\nfor col in df.columns:\n    df[col] = df[col].astype(str)\n\n# Importar librerías\nimport pandas as pd\nfrom sklearn.metrics.pairwise import cosine_similarity\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\n# Crear matriz de características\ntfidf = TfidfVectorizer(stop_words=\'english\')\n\n# Combinar las columnas deseadas en una sola columna\ndf[\'features\'] = df[\'score\'] +\' \'+ df[\'rating\'] +\' \'+ df[\'accion\'] +\' \'+ df[\'drama\'] +\' \'+ df[\'animacion\'] +\' \'+ df[\'documentales\'] +\' \'+ df[\'arte y ciencia\'] +\' \'+ df[\'familia\'] +\' \'+ df[\'internacional\'] +\' \'+ df[\'vida\'] +\' \'+ df[\'comedia\'] +\' \'+ df[\'entretenimiento\'] +\' \'+ df[\'comedia\']+\' \'+ df[\'movie\'] +\' \'+ df[\'tv show\']  +\' \'+ df[\'release_year\'] +\' \'+ df[\'duration_int\'] +\' \'+ df[\'title\'] \n\n# Crear la matriz de características a partir de la columna combinada\ntfidf_matrix = tfidf.f

In [45]:
# Observamos las características utilizadas para recomendar
# df["features"]

In [46]:
# Buscamos la película que colocamos para la recomendación
# df[df["title"] == "marvel studios' iron man 2"]

In [47]:
# Comparamos con el primer resultado de la lista
# df[df["title"] == "marvel studios' iron man"]

In [48]:
#import pickle
# Guardar el modelo en un archivo pickle
#with open('modelo_recomendacion.pkl', 'wb') as f:
#    pickle.dump(cosine_sim, f)


In [49]:
#import pickle
# Cargar el modelo desde el archivo pickle
#with open('modelo_recomendacion.pkl', 'rb') as f:
#    cosine_sim = pickle.load(f)


---

#### En cambio, realizaremos otro modelo mas sencillo y performante. Que también nos da muy buenos resultados

---

In [50]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

def get_recommendations_new(title, df=df, num_recommendations=5):
    features = df.drop(["title","description","id","duration_type"],axis=1).values
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    # Obtener las características de la película a recomendar
    df_filtrado = df[df['title'] == title]
    # Si la película no se encuentra en el conjunto de datos, se crea un vector de características
    caracteristicas = df_filtrado.drop(["title","description","id","duration_type"],axis=1).values
    
    features_filtrado = scaler.transform(caracteristicas)
    
    similarity_matrix_filtrado = cosine_similarity(features_filtrado, features)
    
    sim_scores = list(enumerate(similarity_matrix_filtrado[0]))

    # Ordenar las películas según su similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas más similares
    movie_indices = [i[0] for i in sim_scores[1:num_recommendations+1]]

    peliculas = df[['title',"score"]].iloc[movie_indices].sort_values(by = "score",ascending=False)
    peliculas = list(peliculas["title"])
     # Devolver los títulos de las películas más similares
    return {'recomendacion': peliculas}


In [51]:
get_recommendations_new("marvel studios' iron man")

{'recomendacion': ["marvel studios' captain america: the first avenger",
  'x-men: first class',
  'real steel',
  "marvel studios' iron man 3",
  "marvel studios' thor"]}

In [52]:
get_recommendations_new("toy story 2")

{'recomendacion': ['finding nemo',
  'the lion king 1 1/2',
  'tremors 3: back to perfection',
  'valiant',
  'hercules']}

---